# Operating System
1. Creating new processes:
    * fork() : create new process [Unix Based]
    * exec() : Program loading
    * PCB(Process Control Block) : meta data of the Process
    * <img src="images/fork1.png" width="30%" height="30%" />
    * When sys_fork() is called it creates a new process with the exact code as the parent but with different PID.
    * Parent code will have a `PID of the child` and for the `child process PID == 0.`
    * Execution is resumed in both the processes.
2. Virtual Memory;
    * <img src="images/fork2.png" width="40%" height="40%" />
3. exec():
    * Runs a program present in the file system (that is present in a correct executable format)
    * Creates a new process and the Child code in the Parent code will never be executed after the exec()
    * <img src="images/exec1.png" width="40%" height="40%" />
    * When exec() is called `PID remains the same` along with the file description.
4. File Descriptors:
    * fopen(),fread(), fwrite() - in C are all buffer IO and in turn makes a system call say open(), read() and write() etc.
    * <img src="images/fd1.png" width="40%" height="40%" />
    * The table is in kernel and the only way to access it is using 'fd' below.
    * First 3 slots are booked for console.
    * Accessing FD : fd = open("fodd.txt",O_RDONLY) | N = read(fd, buf, 100)
    * Code <redirect.c>:
        * close(1); // clears the stdout entry in FD
        * #include "kernel/fcntl.h" // to get the macros for the kernel
        * fd = open("hello.out", O_WRONLY| O_CREATE);
        * Read contents of the file, fd = open("hello.out",O_RDONLY); char buf[128]; n = read(fd, buf, 128); buf[n] = '\0';
5. Pipes:
    * A circular buffer to read and write the data into it.
    * Buffer is allocated in the Kernel.
    * Once all the write ends are closed then the read will unblock and continue.
    * <img src="images/pipe1.png" width="40%" height="40%" />
    * p(0) -> read end and p(1) -> write end
    * Code:
        * Important to do `pipe(p)` before fork so that the child and the parent has the pipe entry in the descriptor.
        * <img src="images/pipe2.png" />
        * Setting up > ls | wc 
        * <img src="images/pipe3.png" />
        * <img src="images/pipe4.png" width="40%" height="40%" />
          
6. Adding a Sys Call:
    * Tell us how many free pages are left in the kernel?  Ref `kalloc.c`, `defs.h` and search for // kalloc.c, then changes in `main.c` -> `syscall.h` and add .....(6:18) -> `syscall.c` and add entry ... -> `sysproc.c` ...
    * User side:
        * `user.h` add prototye -> `usys.pl` ...  -> Then create a user program to chech the sys call.
    * Code :
        * <img src="images/Sys1.png" width="75%" height="75%" />
    * Steps:
        * Navigate to `kalloc.c`and add the code you want to make a system call for. `ref int kfreepages()`.
        * Check if the code is correct or not
            * Make an entry in `defs.h` and add an entry under //kalloc.c
            * Navigate to main.c and write a test print() statement.
            * Check if it is working, **cd to xv6 -> make qemu**, if it reaches to booting no errors.
        * Go to syscall.h and add the name of the system call, e.g. **#define SYS_pages  (num)**
        * Go to syscall.c and add the following lines:
            * 107 : extern uint64 sys_pages(void)
            * 131 : [SYS_pages] sys_pages,;
        * There are 2 files were System calls are placed, one is `sysproc` and other is `syscall`
        * Go to sysproc.c and add at the end `uint64 sys_pages(void) {return kfreepages();}`
        * That's all for the kernel side, now let's goto ../user/
        * Open `user.h` and add `int pages(void);` under //system calls
        * Lastly, go to usys.pl and add an entry for `entry("pages");`
        * Add pages.c under the user/ 
        * Navigate to MakeFile and add the entry for pages
        * Check if it is working, **cd to xv6 -> make qemu**, if it reaches to booting no errors.

7. Threads
    * Isolation :
        * <img src="images/iso1.png" width="50%" height="50%" />
    * Map : VA -> Physical Address (Addresses generated by ldr/ str)
    * Virtual Memory:
        * Gives OS the ability to use the disk to extend the amount of RAM(`SWAP`).
        * Not in extensive use now a days due to availibility of large memory
        * COW(Copy On Write), if the process is big, not advisible to copy, fork() : creates a copy of the page table and points to the same physical address and on modification points to new memory address
        * Shared Libraries, used for mapping a single copy of the code(say 'printf') to the process that acutally needs it.
        * Shared Memory, mmap(), to create a chunk of memory for different processes to access.
        * Memory mapped files : created by mmap(), mapping the files to appear just like byte addresses, so the file now appears to be the array of bytes.
        * Dynamically grow the stack.
        * Mapping to memory:
            * <img src="images/vm1.png" width="50%" height="50%" />
        * Page size : 4096 bytes($2^{12}$) | 31.....0 bits, 11....0 [offset], 31....20 [page #] 
        * <img src="images/vm2.png" width="50%" height="50%" />
        * <img src="images/vm3.png" width="25%" height="25%" />
        * Page Table Entry format:
            * <img src="images/pagetable1.png" width="30%" height="30%" />
        * 2 Level Page table:
            * <img src="images/pagetable2.png" width="40%" height="40%" />
            * Thus for getting a data we need 2 memory access, which is very expensive. Hence to minimize this we make use of TLB(Translations Look Aside Buffer). VA -> TLB -> pf# (Page frame #), with TLB having 99.8% hit rate
            * 
            

8. Submission
    * rm -rf xv6-riscv/.git (and no object files)